In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

# import numpy as np
# import re

import apps.matcher as matcher  
# import spacy
import matcher2


In [29]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
# Cargar tokenizador y modelo de HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output.last_hidden_state  # El último estado oculto contiene todas las incrustaciones de tokens
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def obtener_incrustaciones(texto1, texto2):
    # Tokenizar las frases
    encoded_input = tokenizer([texto1, texto2], padding=True, truncation=True, return_tensors='pt')

    # Calcular incrustaciones de tokens
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Realizar Pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalizar las incrustaciones
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    # Calcular la similitud del coseno entre las incrustaciones
    cosine_similarities = cosine_similarity(sentence_embeddings, sentence_embeddings)

    # Escalar los valores de similitud del coseno al rango de 1 a 100
    scaled_similarity = cosine_similarities * 100

    # Redondear los valores para obtener un resultado del 1 al 100
    result = scaled_similarity.round()


    return result[0,1]

def fuzzy_similarity_filter(df, column1, column2, threshold):
    """
    Filtra un DataFrame basado en la similitud de dos columnas utilizando fuzzy matching.

    Args:
        df (DataFrame): El DataFrame que se va a filtrar.
        column1 (str): Nombre de la primera columna.
        column2 (str): Nombre de la segunda columna.
        threshold (int): Umbral de similitud.

    Returns:
        DataFrame: El DataFrame filtrado.
    """
    # with model ia
    filtered_df = df[df.apply(lambda x: obtener_incrustaciones(str(x[column1]), str(x[column2])) >= threshold, axis=1)]
    # with fuzz
    # filtered_df = df[df.apply(lambda x: fuzz.ratio(str(x[column1]), str(x[column2])) >= threshold, axis=1)]

    return filtered_df

In [44]:
# Filtrar el DataFrame utilizando la función fuzzy_similarity_filter
casa = matcher2.get_df('files/match/tokopedia.csv')
casa = casa[casa['error'].isna()]
filtered_df = fuzzy_similarity_filter(casa, 'product_name_lazada', 'product_name_competitor', threshold=75)

# with model ia
filtered_df.to_csv('result_withmodel_tokopedia.csv')
# with fuzz
# filtered_df.to_csv('result_j_lvzoveqs2rdcilf301.csv')
filtered_df

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 6329: character maps to <undefined>

In [9]:
casa[casa['similarity']>=80].to_csv('mayor.csv')

In [2]:
matcher2.harmony_datum_matcher('jecsi', ['product_name_en','product_name_competitor'])

[[2, 0], [2, 1], [2, 2], [2, 5], [2, 6], [2, 7], [2, 9], [2, 10], [2, 11], [2, 12], [2, 13], [13, 0], [13, 1], [13, 2], [13, 5], [13, 6], [13, 7], [13, 9], [13, 10], [13, 11], [13, 12], [13, 13], [25, 0], [25, 1], [25, 2], [25, 5], [25, 6], [25, 7], [25, 9], [25, 10], [25, 11], [25, 12], [25, 13], [27, 0], [27, 1], [27, 2], [27, 5], [27, 6], [27, 7], [27, 9], [27, 10], [27, 11], [27, 12], [27, 13], [38, 0], [38, 1], [38, 2], [38, 5], [38, 6], [38, 7], [38, 9], [38, 10], [38, 11], [38, 12], [38, 13], [44, 0], [44, 1], [44, 2], [44, 5], [44, 6], [44, 7], [44, 9], [44, 10], [44, 11], [44, 12], [44, 13], [52, 0], [52, 1], [52, 2], [52, 5], [52, 6], [52, 7], [52, 9], [52, 10], [52, 11], [52, 12], [52, 13], [61, 0], [61, 1], [61, 2], [61, 5], [61, 6], [61, 7], [61, 9], [61, 10], [61, 11], [61, 12], [61, 13], [78, 0], [78, 1], [78, 2], [78, 5], [78, 6], [78, 7], [78, 9], [78, 10], [78, 11], [78, 12], [78, 13], [88, 0], [88, 1], [88, 2], [88, 5], [88, 6], [88, 7], [88, 9], [88, 10], [88, 11], 

In [5]:
casa[casa['error'].isna()].to_json('jecsi/alternative.json')

In [33]:
import requests
import json

def get_and_split_data_from_api(api_url, params):
    """
    Sends a GET request to the specified API URL with the given parameters,
    retrieves the JSON response, and splits it into two separate dictionaries.

    Args:
        api_url (str): The URL of the API to send the request to.
        params (dict): A dictionary containing the parameters to be sent with the request.

    Returns:
        tuple: A tuple containing two dictionaries, the first dictionary 
        contains the data under the "origin" key, and the second dictionary 
        contains the data under the "alternative" key.
    """
    # Send GET request
    response = requests.get(api_url, params=params)

    # Get JSON response
    r = response.json()

    # Decode the JSON string
    data = json.loads(r["responseData"]["translatedText"])
    print(data)
    # Extract data into separate variables
    origin_data = data["origin"]
    alternative_data = data["alternative"]

    return origin_data, alternative_data

# Example usage:
url = "https://api.mymemory.translated.net/get"

# Parameters of the request
params = {
    "q": '{ "origin": { "product_name": "Official Xiaomi Google TV A 32", "brand_name": "Xiaomi", "color": "Black", "cat_level_1": "Televisions %26 Videos", "cat_level_2": "Smart Televisions" }, "alternative": { "product_name_competitor": "FOR SALE OFFICIAL XIAOMI GOOGLE TV A 32\' TERMURAH - Default", "brand": "Xiaomi", "category_path": "electronic/tv-accessories/television", "color": "" } }',
    "langpair": "id|en"
}

# Retrieve and split data from API
origin, alternative = get_and_split_data_from_api(url, params)
print("Origin Data:")
print(origin)
print("\nAlternative Data:")
print(alternative)


{'origin': {'product_name': 'Official Xiaomi Google TV A 32', 'brand_name': 'Xiaomi', 'color': 'Black', 'cat_level_1': 'Televisions %26 Videos', 'cat_level_2': 'Smart Televisions'}, 'alternative': {'product_name_competitor': "FOR SALE OFFICIAL XIAOMI GOOGLE TV A 32' CHEAPEST - Default", 'brand': 'Xiaomi', 'category_path': 'electronic/tv-accessories/television', 'color': ''}}
Origin Data:
{'product_name': 'Official Xiaomi Google TV A 32', 'brand_name': 'Xiaomi', 'color': 'Black', 'cat_level_1': 'Televisions %26 Videos', 'cat_level_2': 'Smart Televisions'}

Alternative Data:
{'product_name_competitor': "FOR SALE OFFICIAL XIAOMI GOOGLE TV A 32' CHEAPEST - Default", 'brand': 'Xiaomi', 'category_path': 'electronic/tv-accessories/television', 'color': ''}


In [32]:
import requests
import json
# URL de la API

def 
url = "https://api.mymemory.translated.net/get"

# Parámetros de la solicitud
params = {
    "q": '{ "origin": { "product_name": "Official Xiaomi Google TV A 32", "brand_name": "Xiaomi", "color": "Black", "cat_level_1": "Televisions %26 Videos", "cat_level_2": "Smart Televisions" }, "alternative": { "product_name_competitor": "FOR SALE OFFICIAL XIAOMI GOOGLE TV A 32\' TERMURAH - Default", "brand": "Xiaomi", "category_path": "electronic/tv-accessories/television", "color": "" } }',
    "langpair": "id|en"
}

# Realizar la solicitud GET
response = requests.get(url, params=params)

# Imprimir la respuesta
r = response.json()

# Decodificar la cadena JSON
data = json.loads(r["responseData"]["translatedText"])

# Separar en dos variables
origin_data = data["origin"]
alternative_data = data["alternative"]

# Imprimir las variables
print("Datos de origen:")
print(origin_data)
print("\nDatos alternativos:")
print(alternative_data)


Datos de origen:
{'product_name': 'Official Xiaomi Google TV A 32', 'brand_name': 'Xiaomi', 'color': 'Black', 'cat_level_1': 'Televisions %26 Videos', 'cat_level_2': 'Smart Televisions'}

Datos alternativos:
{'product_name_competitor': "FOR SALE OFFICIAL XIAOMI GOOGLE TV A 32' CHEAPEST - Default", 'brand': 'Xiaomi', 'category_path': 'electronic/tv-accessories/television', 'color': ''}


In [31]:
r["responseData"]["translatedText"]

'{ "origin": { "product_name": "Official Xiaomi Google TV A 32", "brand_name": "Xiaomi", "color": "Black", "cat_level_1": "Televisions %26 Videos", "cat_level_2": "Smart Televisions"}, "alternative": {"product_name_competitor": "FOR SALE OFFICIAL XIAOMI GOOGLE TV A 32\' CHEAPEST - Default", "brand": "Xiaomi", "category_path": "electronic/tv-accessories/television", "color": "" } }'

In [37]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Función de Pooling Promedio - Toma en cuenta la máscara de atención para un promedio correcto
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # El último estado oculto contiene todas las incrustaciones de tokens
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Frases para las cuales queremos obtener incrustaciones
# sentences = ['hola', 'hello', 'hola']
sentences = ["Face Care Acne Repair Spot Serum: Reduces acne marks in 3 days 15ml",
 'Buy 1 Take 1 NIVEA Face Care Acne Repair Spot Serum to reduce acne marks 15ml']

# Cargar tokenizador y modelo de HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenizar las frases
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Calcular incrustaciones de tokens
with torch.no_grad():
    model_output = model(**encoded_input)

# Realizar Pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalizar las incrustaciones
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings)

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular la similitud del coseno entre las incrustaciones
cosine_similarities = cosine_similarity(sentence_embeddings, sentence_embeddings)

# Escalar los valores de similitud del coseno al rango de 1 a 100
scaled_similarity = cosine_similarities * 100

# Redondear los valores para obtener un resultado del 1 al 100
result = scaled_similarity.round()

print("Resultado del 1 al 100:")
print(result)

Resultado del 1 al 100:
[[100.  78.]
 [ 78. 100.]]


In [24]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # El último estado oculto contiene todas las incrustaciones de tokens
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def obtener_incrustaciones(texto1, texto2):
    # Cargar tokenizador y modelo de HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
    model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

    # Tokenizar las frases
    encoded_input = tokenizer([texto1, texto2], padding=True, truncation=True, return_tensors='pt')

    # Calcular incrustaciones de tokens
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Realizar Pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalizar las incrustaciones
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    # Calcular la similitud del coseno entre las incrustaciones
    cosine_similarities = cosine_similarity(sentence_embeddings, sentence_embeddings)

    # Escalar los valores de similitud del coseno al rango de 1 a 100
    scaled_similarity = cosine_similarities * 100

    # Redondear los valores para obtener un resultado del 1 al 100
    result = scaled_similarity.round()

    return result[0,1]

# Ejemplo de uso:
texto1 = "SACE LADY Oil Control Powder Waterproof Long-lasting Makeup Setting Powder Easy To Carry Compact Powder And Light Face Makeup Powder With Mirror+Puff"
texto2 = "Loose Powder with Puff - Light"

incrustaciones = obtener_incrustaciones(texto1, texto2)
print(incrustaciones)

52.0


In [28]:
52.1 >= 52

True

In [75]:
cosine_similarities

array([[1.0000004, 0.7517575],
       [0.7517575, 1.       ]], dtype=float32)

In [49]:
from fuzzywuzzy import fuzz
precision = 80
series_origin, series_origin_x=matcher2.get_df('prueba2/origin.json')['title'].astype('str'), matcher2.get_df('prueba2/origin.json')['link'].astype('str')
series_alternative, series_alternative_x=matcher2.get_df('prueba2/alternative_2.json')['product_name'].astype('str'), matcher2.get_df('prueba2/alternative_2.json')['url'].astype('str')
nr = []
for index_row_origin, index_row_origin_x in zip(series_origin, series_origin_x):
    for index_row_alternative, index_row_alternative_x in zip(series_alternative, series_alternative_x):
        if index_row_alternative == 'nan':
            continue
        radio = fuzz.ratio(index_row_origin, index_row_alternative )
        if radio in range(50,79) :
            nr.append({'similitud':radio, 'nombre_origin':index_row_origin.lower(),'link_origin': index_row_origin_x.lower(),'nombre_alternative':index_row_alternative.lower(), 'link_alternative': index_row_alternative_x.lower()    })
            # print(f'{radio} #-# {index_row_origin} #-# {index_row_alternative}')
pd.DataFrame(nr).to_csv('shopee_lazada_range.csv')

In [ ]:
matcher2.get_df('shopee_lazada_range.csv').drop_duplicates()

In [28]:
# Cargar los datos del archivo CSV
df = matcher.get_df('colores.csv')

# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("fr_core_news_sm")
# nlp = spacy.load("es_core_news_sm")

In [15]:
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

In [ ]:
lemmatized_text = " ".join([token.lemma_ for token in df])
doc = nlp("naranja")
for token in doc:
    print(token.text, token.lemma_)

In [71]:
df["word"] = df["word"].map(lambda x: x.lower())
df["lemmatized_text"] = df["word"].apply(lemmatize_text)
df["tag"] = "color"
df["language"] = "fr"

In [29]:
unique_values_by_column = df.apply(lambda col: col.astype(str).unique())

In [ ]:

from sqlalchemy import create_engine
# Establecer una conexión con la base de datos SQLite (si no existe, la creará)
engine = create_engine('sqlite:///data.sqlite', echo=True)

# Guardar el DataFrame en la base de datos (reemplazando si ya existe)
df.to_sql('LemmatizedKeywords', con=engine, if_exists='append', index=False)

In [ ]:
# Consultar la tabla para verificar si se ha cargado correctamente
consulta = pd.read_sql_query('SELECT * FROM LemmatizedKeywords', engine)
print(consulta)

In [66]:
consulta.to_csv("respaldo.csv")

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example product data from two links
product_a = {    "description": "Pink printed Kurta with Trousers with dupatta\n\nKurta design:\nStriped\nA-line shape\nEmpire style\nV-neck, three-quarter regular sleeves\nThread work detail\nCalf length with flared hem\nSilk blend machine weave fabric\n\nTrousers design:\nSolid Trousers\nPartially elasticated waistband\nSlip-on closure\n\nSize & Fit\nThe model (height 5'8) is wearing a size S\n\nMaterial & Care\nTop-Silk Blend\nBottom-Shantoon\nDupatta-Silk Blend\nDry Clean\n\nSpecifications\nSleeve Length\nThree-Quarter Sleeves\nTop Shape\nA-Line\nTop Type\nKurta\nBottom Type\nTrousers\nDupatta\nWith Dupatta\nTop Pattern\nStriped\nTop Design Styling\nEmpire\nTop Hemline\nFlared\nTop Length\nCalf Length\nNeck\nV-Neck\nPrint or Pattern Type\nStriped\nBottom Pattern\nSolid\nBottom Closure\nSlip-On\nWaistband\nPartially Elasticated\nOccasion\nFestive\nOrnamentation\nThread Work\nWeave Pattern\nRegular\nWeave Type\nMachine Weave\nPattern Coverage\nLarge\nDupatta Pattern\nDyed\nDupatta Border\nPrinted\nNumber of Items\n3",
}
product_b = {    "description": """Color may slightly vary due to photography\nBottomwear fabric: shantoon\nDupatta length: 2 m\nDupatta fabric: silk blend\nPackage contains: 1 kurta, 1 dupatta, 1 pants\nKurta, Bottomwear & Dupatta\nDry clean\nMuslin silk\nProduct Code: 466764916002\nMRP: Rs. 7,499.00 inclusive of all taxes(MRP changes as per size selection)\nNet Qty: 1N\nMarketed By: FASHOR LIFESTYLE PVT LTD, 28/30,Sudershan pura, Industrial Area, 22 Godam, Jaipur, Rajasthan, 302006\nManufactured By: FASHOR LIFESTYLE PVT LTD, 28/30,Sudershan pura, Industrial Area, 22 Godam, Jaipur, Rajasthan, 302006\nCountry Of Origin: India\nCustomer Care Address: Tower-B, 7th Floor,IBC Knowledge Park,Bannerghatta Main Road,Bhavani Nagar, S.G. Palya,Bengaluru, Karnataka – 560029,Ph: 1800-889-9991,E-mail: customercare@ajio.com\nCommodity: Women's Kurta Suit Sets",
"""}
# Preprocess and vectorize the data
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([product_a['description'], product_b['description']])

# Calculate similarity
similarity_score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

# Determine if products are a match based on a threshold
threshold = 0.8  # This is an example threshold
print(similarity_score)
if similarity_score[0][0] > threshold:
    print('Products are a match')
else:
    print('Products are not a match')

[[0.12283303]]
Products are not a match


In [2]:
import apps.matcher as matcher

In [3]:
def toolkit_detect_language(file_path: str = None, columns: bool = False)->dict: 
    if file_path is None or len(file_path) == 0:
        raise ValueError("'file_path' cannot be empty")
    if type(file_path) != str:
        raise TypeError("data type compatible only 'str'")
    if type(columns) != bool:
        raise TypeError('Columns must be bool')
    from transformers import pipeline
    pipe = pipeline("text-classification", model="ImranzamanML/GEFS-language-detector")
    max_length = 2187
    def split_text(text, max_length):
        words = text.split()  
        words_selected = []
        currency_length = 0
        for word in words:
            if currency_length + len(word) + sum(map(len, words_selected)) <= max_length:
                words_selected.append(word)
                currency_length += len(word)
            else:
                break       
        return ' '.join(words_selected)
    df = matcher.get_df(file_path)
    if not columns:         
        text_combined = ' '.join(df.apply(lambda row: ', '.join(row.astype(str).unique()), axis=1))
        texto_cortado = split_text(text_combined, max_length)
        return pipe(texto_cortado)[0]['label']  
    # Convert each column to a text series and get unique values
    unique_values_by_column = df.apply(lambda col: col.astype(str).unique())
    # Iterate over the keys of the dictionary of unique values by column
    for column_name, values in unique_values_by_column.items():
        # Join the unique values into a single string
        complete_text = ' '.join(values)       
        # Split the text into segments
        divided_text = split_text(complete_text, max_length) 
        unique_values_by_column[column_name] = pipe(divided_text)[0]['label']
    return unique_values_by_column

'en'

In [59]:
matcher.detect_language_file('files/farmatodo.json')

'The detected language in the CSV is: es'

In [6]:
import os
from apps.ToolsKit import toolkit_detect_language
# import numpy as np

c:\Users\Ivan Rivas\Documents\project\matchX2\.venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [92]:
def Matcherito(file_path_x: str = None, file_path_y: str = None, typematch: str = None, queries: list = []) -> pd.DataFrame:
    
    
    extensions = ['csv', 'json']

    if not any(file_path_x.endswith(extension) for extension in extensions) or not any(file_path_y.endswith(extension) for extension in extensions):
        raise ValueError("The file does not have a valid extension.")

    if file_path_x is None:
        raise ValueError("required field 'file_path_x'")
    elif file_path_y is None:
        raise ValueError("required field 'file_path_y'")
    else:
        if file_path_x == file_path_y:
            raise ValueError("Cannot put the same file on both paths")
        if not os.path.exists(file_path_x) or not os.path.exists(file_path_y):
            raise FileNotFoundError("required file does not exist in the path")

    if typematch is None:
        raise ValueError("required field 'typematch'")
    elif not typematch in ['exact', 'similar']:#cambiar a variable de entorno con la lista de tipo de match
        raise ValueError("match type not valid")
    
    # if not language in ['es', 'en', 'fr']:
        # raise ValueError("unsupported language")

    if type(file_path_x) != str or type(file_path_y) != str or type(typematch) != str or type(queries) != list:
        raise TypeError("Data type error in one of the attributes")
    
    if toolkit_detect_language(file_path_x) != toolkit_detect_language(file_path_y):
        raise TypeError("The language must be the same")


    #start logic for match
    if len(queries) == 0:
        # if not language['all'] is  None:
            # print('hola')
            # pass #detect language of file
        
        

        pass #run script auto
    else:
        # run script manual

        # Perform data matching based on the query
        if np.ndim(queries) == 1:
            # Harmonize indices and get matched data
            data_match = harmonize_index_match(df_origin, df_alternative, query)
            print(data_match)
            # Create a DataFrame for the matched data
            df_alternative_result = pd.DataFrame(columns=df_alternative.columns)
            df_origin_result = pd.DataFrame(columns=df_origin.columns)

            for row in data_match:
                # print(df_alternative.iloc[row[1]])
                # df_origin_result.loc[len(df_origin_result.index)] = df_origin.iloc[row[0]]

                df_alternative_result.loc[len(df_alternative_result.index)] = df_alternative.iloc[row[1]]

                # df_alternative_result.loc[len(df_alternative_result.index)] = df_alternative.iloc[row[1]]
            # Save the result to JSON with or without duplicates
                
            # df_alternative_result = df_alternative_result.merge(df_origin_result,how='cross')
            if with_out_duplicates == 0:
                df_alternative_result.to_json(f'{pathFiles}/result_withDuplicates_simple.json')
                # df_origin_result.to_json(f'{pathFiles}/Oresult_withDuplicates_simple.json')
            else:
                df_alternative_result.drop_duplicates().to_json(f'{pathFiles}/result_withOutDuplicates_simple.json')
                # df_origin_result.drop_duplicates().to_json(f'{pathFiles}/Oresult_withOutDuplicates_simple.json')

        else:
             # Multiple queries case
        
            if len(query) >= 2:
                # Harmonize indices for each query and store the results in a matrix
                matriz_data_match = []
                for query_x in query:
                    matriz_data_match.append(harmonize_index_match(df_origin, df_alternative, query_x, precision=70))
                print(matriz_data_match)
                # Convert the matrix to a dictionary
                variables = arr_to_dic(matriz_data_match)
                
                # Perform matching on pairs of columns
                result = matcher_pair_columns(variables)
                
                # Save the result to JSON with or without duplicates
                if with_out_duplicates == 0:
                    df_result = df_alternative.iloc[result].reset_index(drop=True)
                    # print(df_result)
                    df_result.to_json(f'{pathFiles}/result_withDuplicates_multi.json')
                else:
                    df_alternative.iloc[result].drop_duplicates().to_json(f'{pathFiles}/result_withOutDuplicates_multi.json')

In [ ]:
Matcherito('colores.csv','respaldo.csv', 'exact')